## 回歸分析入門 - 第一課

#### 放到情境中來看

✅ 回歸方法有很多種類，選擇哪一種取決於你想要解答的問題。如果你想預測某個年齡的人可能的身高，你會使用 `線性回歸`，因為你在尋找一個**數值**。如果你想了解某種菜餚是否應該被視為純素，你在尋找的是一個**類別分配**，因此你會使用 `邏輯回歸`。稍後你會學到更多關於邏輯回歸的內容。試著思考一些你可以向數據提出的問題，以及哪種方法更適合回答這些問題。

在本節中，你將使用一個[關於糖尿病的小型數據集](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html)。想像一下，你想要測試一種針對糖尿病患者的治療方法。機器學習模型可能幫助你判斷哪些患者基於變數的組合會對治療有更好的反應。即使是一個非常基礎的回歸模型，當可視化時，也可能顯示出一些關於變數的信息，幫助你組織理論上的臨床試驗。

說到這裡，讓我們開始這項任務吧！

<p >
   <img src="../../images/encouRage.jpg"
   width="630"/>
   <figcaption>插畫作者：@allison_horst</figcaption>

<!--![插畫作者：\@allison_horst](../../../../../../translated_images/encouRage.e75d5fe0367fb9136b78104baf4e2032a7622bc42a2bc34c0ad36c294eeb83f5.tw.jpg)<br>插畫作者：@allison_horst-->


## 1. 加載工具集

在這項任務中，我們需要以下套件：

-   `tidyverse`： [tidyverse](https://www.tidyverse.org/) 是一個[由 R 套件組成的集合](https://www.tidyverse.org/packages)，旨在讓數據科學更快速、更簡單、更有趣！

-   `tidymodels`： [tidymodels](https://www.tidymodels.org/) 框架是一個[由套件組成的集合](https://www.tidymodels.org/packages)，用於建模和機器學習。

您可以使用以下指令安裝它們：

`install.packages(c("tidyverse", "tidymodels"))`

以下腳本會檢查您是否擁有完成此模組所需的套件，並在缺少某些套件時為您安裝。


In [2]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))
pacman::p_load(tidyverse, tidymodels)

Loading required package: pacman



現在，讓我們載入這些很棒的套件並使它們在我們目前的 R 工作環境中可用。（這僅是為了說明，`pacman::p_load()` 已經為您完成了這項工作）


In [ ]:
# load the core Tidyverse packages
library(tidyverse)

# load the core Tidymodels packages
library(tidymodels)


## 2. 糖尿病數據集

在這個練習中，我們將展示回歸分析的技能，並使用糖尿病數據集進行預測。[糖尿病數據集](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.rwrite1.txt) 包含 `442 個樣本` 的糖尿病相關數據，包括 10 個預測特徵變數：`年齡`、`性別`、`身體質量指數`、`平均血壓` 和 `六項血清測量值`，以及一個結果變數 `y`：基準一年後疾病進展的定量測量。

|觀測數量|442|
|----------------------|:---|
|預測變數數量|前 10 列為數值型預測變數|
|結果/目標|第 11 列為基準一年後疾病進展的定量測量|
|預測變數資訊|- 年齡（以年計算）
||- 性別
||- bmi 身體質量指數
||- bp 平均血壓
||- s1 tc，總血清膽固醇
||- s2 ldl，低密度脂蛋白
||- s3 hdl，高密度脂蛋白
||- s4 tch，總膽固醇 / HDL
||- s5 ltg，可能是血清甘油三酯水平的對數
||- s6 glu，血糖水平|

> 🎓 記住，這是監督式學習，我們需要一個名為 'y' 的目標變數。

在使用 R 操作數據之前，您需要將數據導入 R 的內存，或者建立一個 R 可以用來遠程訪問數據的連接。

> [readr](https://readr.tidyverse.org/) 套件是 Tidyverse 的一部分，它提供了一種快速且友好的方式將矩形數據讀入 R。

現在，讓我們加載來自以下來源 URL 的糖尿病數據集：<https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html>

此外，我們將使用 `glimpse()` 對數據進行基本檢查，並使用 `slice()` 顯示前 5 行。

在繼續之前，讓我們介紹一個您在 R 代碼中經常會遇到的東西 🥁🥁：管道運算符 `%>%`

管道運算符 (`%>%`) 通過將一個對象向前傳遞到函數或調用表達式中，按邏輯順序執行操作。您可以將管道運算符理解為代碼中的 "然後"。


In [ ]:
# Import the data set
diabetes <- read_table2(file = "https://www4.stat.ncsu.edu/~boos/var.select/diabetes.rwrite1.txt")


# Get a glimpse and dimensions of the data
glimpse(diabetes)


# Select the first 5 rows of the data
diabetes %>% 
  slice(1:5)

`glimpse()` 告訴我們這份資料有 442 行和 11 列，所有的列都是 `double` 資料型別。

<br>

> glimpse() 和 slice() 是 [`dplyr`](https://dplyr.tidyverse.org/) 中的函數。Dplyr 是 Tidyverse 的一部分，是一種資料操作的語法，提供了一組一致的動詞，幫助解決最常見的資料操作挑戰。

<br>

現在我們已經有了資料，讓我們聚焦到其中一個特徵（`bmi`），作為這次練習的目標。這需要我們選擇所需的列。那麼，我們該如何做到呢？

[`dplyr::select()`](https://dplyr.tidyverse.org/reference/select.html) 允許我們在資料框中*選擇*（並可選擇性地重新命名）列。


In [ ]:
# Select predictor feature `bmi` and outcome `y`
diabetes_select <- diabetes %>% 
  select(c(bmi, y))

# Print the first 5 rows
diabetes_select %>% 
  slice(1:10)

## 3. 訓練與測試資料

在監督式學習中，通常會將資料*分割*成兩個子集；一個（通常較大的）用來訓練模型的集合，以及一個較小的「保留」集合，用來檢視模型的表現。

現在我們已經準備好資料，可以看看機器是否能幫助我們在這個數據集中找到一個合理的分割方式。我們可以使用 [rsample](https://tidymodels.github.io/rsample/) 套件，這是 Tidymodels 框架的一部分，來建立一個包含如何分割資料資訊的物件，然後使用另外兩個 rsample 函數來提取生成的訓練和測試集合：


In [ ]:
set.seed(2056)
# Split 67% of the data for training and the rest for tesing
diabetes_split <- diabetes_select %>% 
  initial_split(prop = 0.67)

# Extract the resulting train and test sets
diabetes_train <- training(diabetes_split)
diabetes_test <- testing(diabetes_split)

# Print the first 3 rows of the training set
diabetes_train %>% 
  slice(1:10)

## 4. 使用 Tidymodels 訓練線性回歸模型

現在我們準備好訓練模型了！

在 Tidymodels 中，您可以使用 `parsnip()` 來指定模型，並定義以下三個概念：

-   模型**類型**區分不同的模型，例如線性回歸、邏輯回歸、決策樹模型等。

-   模型**模式**包括常見選項如回歸和分類；某些模型類型支持這兩種模式，而有些僅支持其中一種。

-   模型**引擎**是用來擬合模型的計算工具。通常這些是 R 套件，例如 **`"lm"`** 或 **`"ranger"`**。

這些建模資訊會被捕捉到模型規範中，所以讓我們來建立一個吧！


In [ ]:
# Build a linear model specification
lm_spec <- 
  # Type
  linear_reg() %>% 
  # Engine
  set_engine("lm") %>% 
  # Mode
  set_mode("regression")


# Print the model specification
lm_spec

在模型被*指定*之後，可以使用 [`fit()`](https://parsnip.tidymodels.org/reference/fit.html) 函數來進行模型的`估算`或`訓練`，通常需要使用公式和一些數據。

`y ~ .` 的意思是我們將 `y` 作為預測的量/目標，由所有的預測變數/特徵來解釋，也就是 `.`（在這個例子中，我們只有一個預測變數：`bmi`）。


In [ ]:
# Build a linear model specification
lm_spec <- linear_reg() %>% 
  set_engine("lm") %>%
  set_mode("regression")


# Train a linear regression model
lm_mod <- lm_spec %>% 
  fit(y ~ ., data = diabetes_train)

# Print the model
lm_mod

從模型輸出中，我們可以看到訓練過程中學到的係數。這些係數代表最佳擬合線的係數，該線能使實際變數與預測變數之間的整體誤差降到最低。
<br>

## 5. 在測試集上進行預測

現在我們已經訓練了一個模型，可以使用 [parsnip::predict()](https://parsnip.tidymodels.org/reference/predict.model_fit.html) 來對測試數據集的疾病進展 y 進行預測。這將用於繪製數據組之間的分界線。


In [ ]:
# Make predictions for the test set
predictions <- lm_mod %>% 
  predict(new_data = diabetes_test)

# Print out some of the predictions
predictions %>% 
  slice(1:5)

哇哦！💃🕺 我們剛剛訓練了一個模型並用它進行了預測！

在進行預測時，tidymodels 的慣例是始終生成一個具有標準化列名的 tibble/資料框結果。這使得將原始數據和預測結果結合成一個可用格式變得非常簡單，方便後續操作，例如繪圖。

`dplyr::bind_cols()` 可以高效地將多個資料框的列綁定在一起。


In [ ]:
# Combine the predictions and the original test set
results <- diabetes_test %>% 
  bind_cols(predictions)


results %>% 
  slice(1:5)

## 6. 繪製模型結果

現在是時候用視覺化來呈現了 📈。我們將建立一個散佈圖，展示測試集中的所有 `y` 和 `bmi` 值，然後使用預測結果繪製一條線，將模型的數據分組以最合適的方式連接起來。

R 有多種繪圖系統，但 `ggplot2` 是其中最優雅且最具靈活性的。它允許你通過**組合獨立的組件**來構建圖表。


In [ ]:
# Set a theme for the plot
theme_set(theme_light())
# Create a scatter plot
results %>% 
  ggplot(aes(x = bmi)) +
  # Add a scatter plot
  geom_point(aes(y = y), size = 1.6) +
  # Add a line plot
  geom_line(aes(y = .pred), color = "blue", size = 1.5)

✅ 想一想這裡發生了什麼。一條直線穿過許多小數據點，但它究竟在做什麼？你能否看出如何利用這條直線來預測一個新的、未見過的數據點應該如何與圖表的 y 軸產生關聯？試著用語言描述這個模型的實際用途。

恭喜你！你已經建立了你的第一個線性回歸模型，使用它進行了預測，並在圖表中展示了結果！



---

**免責聲明**：  
本文件使用 AI 翻譯服務 [Co-op Translator](https://github.com/Azure/co-op-translator) 進行翻譯。儘管我們致力於提供準確的翻譯，請注意自動翻譯可能包含錯誤或不準確之處。原始文件的母語版本應被視為權威來源。對於重要資訊，建議尋求專業人工翻譯。我們對因使用此翻譯而產生的任何誤解或錯誤解釋不承擔責任。
